# Link USGS Gages to NWM 3.0 Reach IDs

Given USGS gage IDs, use the National Water Model RouteLink topology file to link gages to NWM reaches.

- `camels_name.txt` sourced from https://ral.ucar.edu/solutions/products/camels.
- `RouteLink_CONUS.nc` sourced from https://www.nco.ncep.noaa.gov/pmb/codes/nwprod/nwm.v3.0.13/parm/domain/RouteLink_CONUS.nc 
- `hfv3_conuscats.parquet` and `hfv3_conusreaches.parquet` adapted from https://water.noaa.gov/resources/downloads/nwm/NWM_channel_hydrofabric.tar.gz

Note: the source for RouteLink may change as the NWM gets updated by NOAA-OWP. Previous versions are not available for download.

The expected output of this notebook is `camels_link.csv`.

Written by Quinn Lee (GitHub @quinnylee)

In [9]:
import xarray as xr
import pandas as pd

In [10]:
# Read USGS IDs from CAMELS names file

with open("camels_name.txt", "r") as gagesfile:
    gagesfile.readline()
    lines = gagesfile.readlines()
    gageslist = []
    for line in lines:
        linelist = line.split(';')
        gageslist.append(linelist[0])

gageslist

['01013500',
 '01022500',
 '01030500',
 '01031500',
 '01047000',
 '01052500',
 '01054200',
 '01055000',
 '01057000',
 '01073000',
 '01078000',
 '01118300',
 '01121000',
 '01123000',
 '01134500',
 '01137500',
 '01139000',
 '01139800',
 '01142500',
 '01144000',
 '01162500',
 '01169000',
 '01170100',
 '01181000',
 '01187300',
 '01195100',
 '01333000',
 '01350000',
 '01350080',
 '01350140',
 '01365000',
 '01411300',
 '01413500',
 '01414500',
 '01415000',
 '01423000',
 '01434025',
 '01435000',
 '01439500',
 '01440000',
 '01440400',
 '01451800',
 '01466500',
 '01484100',
 '01485500',
 '01486000',
 '01487000',
 '01491000',
 '01510000',
 '01516500',
 '01518862',
 '01532000',
 '01539000',
 '01542810',
 '01543000',
 '01543500',
 '01544500',
 '01545600',
 '01547700',
 '01548500',
 '01549500',
 '01550000',
 '01552000',
 '01552500',
 '01557500',
 '01567500',
 '01568000',
 '01580000',
 '01583500',
 '01586610',
 '01591400',
 '01594950',
 '01596500',
 '01605500',
 '01606500',
 '01613050',
 '01620500',

In [11]:
routelink_ds = xr.open_dataset("../RouteLink_CONUS.nc")
routelink_ds

<xarray.Dataset> Size: 269MB
Dimensions:            (feature_id: 2776734)
Coordinates:
    lon                (feature_id) float32 11MB ...
    lat                (feature_id) float32 11MB ...
Dimensions without coordinates: feature_id
Data variables: (12/21)
    link               (feature_id) int32 11MB ...
    from               (feature_id) int32 11MB ...
    to                 (feature_id) int32 11MB ...
    alt                (feature_id) float32 11MB ...
    order              (feature_id) int32 11MB ...
    Qi                 (feature_id) float32 11MB ...
    ...                 ...
    gages              (feature_id) |S15 42MB ...
    Kchan              (feature_id) int16 6MB ...
    ascendingIndex     (feature_id) int32 11MB ...
    nCC                (feature_id) float32 11MB ...
    TopWdthCC          (feature_id) float32 11MB ...
    TopWdth            (feature_id) float32 11MB ...
Attributes:
    Convention:        CF-1.6
    featureType:       timeSeries
    history:           Created Thu Sep  9 18:11:34 2021
    processing_notes:  This file was produced Thu Sep  9 16:16:38 2021 by Kev...

In [12]:
# Subset dataset to useful information
subslice = [
    "link",
    "to",
    "gages",
    "Length"
]
routelink_df = routelink_ds[subslice].to_dataframe().astype({"link": int, "to": int,})
routelink_df = routelink_df.set_index("link")

In [13]:
# Turn gages into strings and add to routelink dataframe
gagestr = []
for i in range(len(routelink_df)):
    gagestr.append(str(routelink_df['gages'].iloc[i]))
routelink_df['gagestr'] = gagestr
routelink_df.head()

,to,gages,Length,lon,lat,gagestr
link,,,,,,
6635572,6635570,b' ',1070.0,-96.540199,46.228783,b' '
6635590,6635600,b' ',1117.0,-96.530647,46.213486,b' '
6635598,6635636,b' ',2303.0,-96.505341,46.201508,b' '
6635622,6635620,b' ',1119.0,-96.615021,46.200523,b' '
6635626,6635624,b' ',3171.0,-96.637161,46.195522,b' '


In [14]:
# Filter to only include gages in the CAMELS dataset
serieslist = []
for gage in gageslist:
    serieslist.append(routelink_df[routelink_df['gagestr'].str.contains(gage)])

camels_df = pd.concat(serieslist)
camels_df.head()

,to,gages,Length,lon,lat,gagestr
link,,,,,,
724696,724684,b' 01013500',2257.0,-68.577705,47.233467,b' 01013500'
2677104,2678276,b' 01022500',498.0,-67.933594,44.609032,b' 01022500'
3923,6087,b' 01030500',1616.0,-68.299759,45.505806,b' 01030500'
1722317,1722313,b' 01031500',2684.0,-69.330780,45.172615,b' 01031500'
3321976,3322000,b' 01047000',3052.0,-69.963158,44.874397,b' 01047000'


In [15]:
camels_df.to_csv("camels_link.csv")